# Reinforcement Learning for Navigation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv, MultiAgentVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor



from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper, UnityToMultiGymWrapper
import gym_unity
import numpy as np

In [3]:
ENV_PATH = r"C:\Users\batua\Desktop\RLNav\NavigationEnvironments\Debug\Ball3D\UnityEnvironment.exe"

In [4]:
# for _ in range(1024):
#     action = np.random.uniform(low=-2, high=2, size=(16,2))
#     multi_env.step(action)

In [5]:
action = np.random.uniform(low=-1, high=1, size=(0,2))

In [6]:
import random, os

In [7]:
log_dir = "tmp/"
os.makedirs(log_dir, exist_ok=True)

def make_env(rank, seed=0):
    def _init():
        unity_env = UnityEnvironment(ENV_PATH, base_port=5000 + random.randint(0,1000) + rank)
        env = UnityToMultiGymWrapper(unity_env)
        env.seed(seed + rank)
        env = Monitor(env, log_dir)
        return env
    set_random_seed(seed)
    return _init


In [8]:
breaker = random.randint(0,1000)
env = MultiAgentVecEnv(make_env(0, breaker), 12)

2021-04-22 19:36:44 INFO [environment.py:113] Connected to Unity environment with package version 1.9.0-preview and communication version 1.5.0
2021-04-22 19:36:44 INFO [environment.py:282] Connected new brain:
3DBall?team=0
2021-04-22 19:36:44 WARNING [__init__.py:106] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
C:\Users\batua\anaconda3\envs\rlnav\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2021-04-22 19:36:44 WARNING [__init__.py:336] Could not seed environment 3DBall?team=0


12
num envs in multiagentvecenv 12


In [9]:
env.reset()

array([[-4.58385758e-02, -2.70688180e-02, -1.43874550e+00,
         3.99607611e+00,  9.79341984e-01,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [ 7.30369315e-02, -7.14064389e-02, -9.66649055e-01,
         3.99607611e+00,  1.29474640e-01,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [ 4.42208499e-02, -7.51109868e-02,  1.27105904e+00,
         3.99607563e+00,  9.11190510e-01,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [ 4.76103649e-02,  2.27974961e-03,  1.14177227e+00,
         3.99607611e+00,  3.54968071e-01,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [ 5.12608923e-02,  2.75232233e-02, -6.70847893e-01,
         3.99607563e+00,  1.00081205e+00,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [-4.58375644e-03,  1.59135635e-03,  1.07326746e-01,
         3.99607563e+00, -1.20461130e+00,  0.00000000e+00,
        -1.96199998e-01,  0.00000000e+00],
       [-1.12987794e-02, -2.171101

In [12]:
env.reset()
for _ in range(128):
    action = np.random.uniform(low=-1, high=1, size=(12,2))
    env.step(action)

In [ ]:
model = PPO('MlpPolicy', env, verbose=2)
model.learn(total_timesteps=1000000, )

Using cuda device
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 39.5         |
|    ep_rew_mean          | 3.94         |
| time/                   |              |
|    fps                  | 3128         |
|    iterations           | 1            |
|    time_elapsed         | 7            |
|    total_timesteps      | 24576        |
| train/                  |              |
|    approx_kl            | 0.0038244352 |
|    clip_fraction        | 0.0369       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.85        |
|    explained_variance   | 0.0512       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.888        |
|    n_updates            | 50           |
|    policy_gradient_loss | -0.00193     |
|    std                  | 1.01         |
|    value_loss           | 1.1          |
------------------------------------------
------------------------------------

In [ ]:
obs = env.reset()
print(obs.shape)
for _ in range(1024):
    actions = [model.predict(ob) for ob in obs]
    print(np.array(actions).shape)
    env.step(actions)

In [ ]:
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)